In [2]:
# Header block to include all modules that must be imported ahead of time
# Only needs to be run once per session, and each time a new module is added

import requests
import json
import pandas as pd # this will need to be installed via command line first
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [9]:
# Method to get the salary cap hits for every player currently signed on an NFL team
# Creates a csv with columns player name, salary cap hit, and team location abbrviation 

def GetTeamSalary(team: list): # accepts a list ['team-name', 'LOC']
    # Use requests to get the raw HTML response from overthecap.com
    OTCResponse = requests.get(f'https://overthecap.com/salary-cap/{team[0]}')
        
    # If we get a normal response proceed with data scraping
    if OTCResponse.status_code == 200:
        print(f"Request succeeded with status code {OTCResponse.status_code}")
        TeamNameCapitalized = (team[0].replace("-", " ")).title()
        print(f"Here are the Salary Cap hits for all players who are currently signed with the {TeamNameCapitalized}:")

        # Pandas read_html method allows the table contents to be put into a DataFrame
        TableTag = 'salary-cap-table contracted-players'
        DataFrames = pd.read_html(OTCResponse.content, attrs = {'class': TableTag})
        
        # Select the specific parts of the dataframe we want 
        SalariesDF = DataFrames[0][['Player', 'Cap Number']]
        if len(SalariesDF) >= 51:
            SalariesDF = SalariesDF.drop(51, axis=0)
        # Add team location abv to each player
        SalariesDF = SalariesDF.assign(TeamLOC = team[1])
        
        # Change the order of columns to make more sense semantically 
        new_order = ['Player', 'TeamLOC','Cap Number']
        SalariesDF = SalariesDF.loc[:, new_order]
        SalariesDF.columns = ['Player', 'Team Location', 'Salary Cap Hit']
        # Sort dataframe by player name
        SalariesDF = SalariesDF.sort_values(by=['Player'], ignore_index=True)
        
        # Specify filename and path
        csvPathName = f'./data/team-salary-lists/{TeamNameCapitalized}-Player-Salary-List.csv'
        
        # Create a .csv file with all the player names and their cap hits
        SalariesDF.to_csv(csvPathName, index = False, header = False)
        
        # Read the csv back for testing purposes
        SalariesCSV = pd.read_csv(f'./data/team-salary-lists/{TeamNameCapitalized}-Player-Salary-List.csv', header = None)
        SalariesCSV.columns = ['Player', 'Team Location', 'Salary Cap Hit']
        print(SalariesCSV)
        
    # If we don't get a normal reponse, stop scraping
    else:
        print(f"Request failed with status code {OTCResponse.status_code}")
        return -1


In [10]:
# Method to get the salary cap hits for every player currently signed on every NFL team
# Reads in the team list csv, and calls the GetTeamSalaryCapHits method for each team
def GetAllTeamSalary():
    # Read in the full team list csv
    FullTeamListCSV = pd.read_csv('./data/team-attribute-lists/Teams-List.csv', header = None)
    FullTeamListCSV.columns = ['TeamName', 'Abbreviation', 'Conference', 'Division']
    
    # Create a list of all the team names and abbreviations as a list of lists
    TeamList = []
    FullTeamListCSV.apply(lambda row: TeamList.append(row[:2].tolist()), axis = 1)
    
    # Get the salary cap hits for each team
    for team in TeamList:
        # format the team name to be used in the URL
        team[0] = team[0].replace(" ", "-").lower()
        GetTeamSalary(team) # team = ['team-name', 'LOC']
    

GetAllTeamSalary()

Request succeeded with status code 200
Here are the Salary Cap hits for all players who are currently signed with the Arizona Cardinals:
                     Player Team Location Salary Cap Hit
0              Aaron Brewer           ARI     $1,152,500
1           Andre Baccellia           ARI       $985,000
2            Andre Chachere           ARI     $1,055,000
3          Austen Pleasants           ARI       $795,000
4              B.J. Ojulari           ARI     $1,954,003
5                Ben Stille           ARI       $985,000
6        Bernhard Seikovits           ARI       $795,000
7             Bilal Nichols           ARI     $5,005,000
8            Blake Gillikin           ARI     $1,600,000
9          Blake Whiteheart           ARI       $915,000
10              Bobby Price           ARI     $1,060,294
11              Budda Baker           ARI    $19,025,000
12           Cameron Thomas           ARI     $1,403,002
13         Carter O'Donnell           ARI       $915,000
14      

Request succeeded with status code 200
Here are the Salary Cap hits for all players who are currently signed with the Baltimore Ravens:
                  Player Team Location Salary Cap Hit
0            Adisa Isaac           BAL     $1,026,915
1         Andrew Vorhees           BAL       $820,528
2   Ar'Darius Washington           BAL       $985,000
3          Arthur Maulet           BAL     $1,500,000
4             Beau Brade           BAL       $803,333
5          Ben Cleveland           BAL     $1,500,460
6       Brandon Stephens           BAL     $3,326,154
7            Brent Urban           BAL     $1,152,500
8   Broderick Washington           BAL     $4,540,000
9          C.J. Ravenell           BAL       $795,000
10         Charlie Kolar           BAL     $1,161,220
11           Chris Board           BAL     $1,152,500
12         Chris Collier           BAL       $796,666
13     Christian Matthew           BAL       $985,000
14         Corey Bullock           BAL       $796,166


Request succeeded with status code 200
Here are the Salary Cap hits for all players who are currently signed with the Carolina Panthers:
                  Player Team Location Salary Cap Hit
0       A'Shawn Robinson           CAR     $3,290,000
1           Adam Thielen           CAR     $9,917,000
2              Alex Cook           CAR       $915,000
3            Amare Barno           CAR     $1,031,352
4            Andrew Raym           CAR       $800,000
5            Andy Dalton           CAR     $4,967,000
6         Austin Corbett           CAR    $10,133,000
7          Badara Traore           CAR       $795,000
8      Brady Christensen           CAR     $3,394,778
9            Bryce Young           CAR     $8,626,153
10             Cade Mays           CAR     $1,028,267
11              Cam Gill           CAR     $1,055,000
12              Cam Sims           CAR       $985,000
13       Chandler Wooten           CAR       $985,000
14       Chandler Zavala           CAR     $1,107,934

Request succeeded with status code 200
Here are the Salary Cap hits for all players who are currently signed with the Cincinnati Bengals:
                 Player Team Location Salary Cap Hit
0           Aaron Casey           CIN       $800,000
1   Akeem Davis-Gaither           CIN     $2,544,382
2            Alex Cappa           CIN     $9,250,000
3          Allan George           CIN       $915,000
4          Amarius Mims           CIN       $795,000
5        Andrei Iosivas           CIN       $954,848
6       Austin McNamara           CIN       $800,000
7             B.J. Hill           CIN    $10,833,334
8          Brad Robbins           CIN       $948,333
9          Cal Adomitis           CIN       $985,000
10           Cam Grandy           CIN       $796,666
11     Cam Taylor-Britt           CIN     $1,623,472
12       Cameron Sample           CIN     $1,252,037
13       Cedric Johnson           CIN       $829,597
14        Charlie Jones           CIN     $1,084,694
15          Ch

Request succeeded with status code 200
Here are the Salary Cap hits for all players who are currently signed with the Dallas Cowboys:
                  Player Team Location Salary Cap Hit
0            Alec Holler           DAL       $795,000
1          Asim Richards           DAL       $976,245
2          Brandin Cooks           DAL    $10,000,000
3         Brandon Aubrey           DAL       $915,000
4      Brevyn Spann-Ford           DAL       $801,666
5          Brock Hoffman           DAL       $985,000
6         Brock Mogensen           DAL       $796,000
7            Bryan Anger           DAL     $3,800,000
8          Buddy Johnson           DAL       $915,000
9          Byron Vaughns           DAL       $796,666
10          C.J. Goodwin           DAL     $1,152,500
11         Caelen Carson           DAL       $858,569
12           Cam Johnson           DAL       $798,333
13            Carl Davis           DAL     $1,068,500
14           CeeDee Lamb           DAL    $17,991,000
15

...

Request succeeded with status code 200
Here are the Salary Cap hits for all players who are currently signed with the New Orleans Saints:
                   Player Team Location Salary Cap Hit
0              A.T. Perry            NO       $959,091
1           Adam Prentice            NO     $1,130,000
2          Alontae Taylor            NO     $1,964,917
3            Alvin Kamara            NO    $18,553,824
4           Anfernee Orji            NO       $795,000
5             Blake Grupe            NO       $915,833
6            Bryan Bresee            NO     $2,787,171
7               Bub Means            NO       $858,569
8          Cameron Jordan            NO    $13,852,250
9         Carl Granderson            NO     $5,250,000
10     Cedrick Wilson Jr.            NO     $1,850,000
11             Cesar Ruiz            NO     $4,450,000
12          Charlie Smyth            NO       $795,000
13            Chase Young            NO     $3,442,000
14            Chris Olave            

Request succeeded with status code 200
Here are the Salary Cap hits for all players who are currently signed with the New York Jets:
                      Player Team Location Salary Cap Hit
0              Aaron Rodgers           NYJ    $17,161,112
1              Ahmad Gardner           NYJ     $9,122,945
2              Al Blades Jr.           NYJ       $798,333
3         Alijah Vera-Tucker           NYJ     $5,054,329
4               Allen Lazard           NYJ    $12,184,000
5             Andrew Peasley           NYJ       $795,000
6               Ashtyn Davis           NYJ     $1,292,500
7             Austin Deculus           NYJ       $985,000
8             Austin Seibert           NYJ       $985,000
9               Brady Latham           NYJ       $797,500
10             Braelon Allen           NYJ       $927,177
11          Braiden McGregor           NYJ       $801,666
12            Brandin Echols           NYJ     $1,098,548
13        Brandon Codrington           NYJ       $795,0

Request succeeded with status code 200
Here are the Salary Cap hits for all players who are currently signed with the Pittsburgh Steelers:
                   Player Team Location Salary Cap Hit
0         Aaron Shampklin           PIT       $795,000
1          Alex Highsmith           PIT     $6,727,000
2          Anderson Hardy           PIT       $795,000
3         Anthony Averett           PIT       $985,000
4           Beanie Bishop           PIT       $803,333
5          Breiden Fehoko           PIT     $1,055,000
6         Broderick Jones           PIT     $3,778,719
7       Calvin Austin III           PIT     $1,111,084
8         Cameron Heyward           PIT    $22,406,250
9        Cameron Johnston           PIT     $2,083,333
10         Cameron Sutton           PIT       $985,000
11          Chris Boswell           PIT     $4,720,000
12        Christian Kuntz           PIT     $1,163,333
13           Cole Holcomb           PIT     $7,640,000
14         Connor Heyward           

Request succeeded with status code 200
Here are the Salary Cap hits for all players who are currently signed with the Seattle Seahawks:
                 Player Team Location Salary Cap Hit
0           A.J. Barner           SEA       $983,777
1         Abraham Lucas           SEA     $1,470,456
2       Andrew Whitaker           SEA       $795,000
3      Anthony Bradford           SEA     $1,115,997
4           Artie Burns           SEA     $1,152,500
5             Boye Mafe           SEA     $2,335,162
6         Brady Russell           SEA       $915,000
7          Byron Murphy           SEA     $2,924,204
8         Cameron Young           SEA     $1,096,877
9       Carlton Johnson           SEA       $796,666
10        Charles Cross           SEA     $5,831,898
11          Chris Stoll           SEA       $917,166
12     Christian Haynes           SEA     $1,055,047
13          Coby Bryant           SEA     $1,186,952
14           Cody White           SEA       $915,000
15     D'Wayne E

Request succeeded with status code 200
Here are the Salary Cap hits for all players who are currently signed with the Tennessee Titans:
                   Player Team Location Salary Cap Hit
0            Amani Hooker           TEN     $9,317,647
1          Andrew Rupcich           TEN       $915,000
2         Anthony Kendall           TEN       $915,000
3               Arden Key           TEN     $4,220,000
4        Brayden Narveson           TEN       $796,666
5            Brian Dooley           TEN       $798,333
6            Bryce Oliver           TEN       $795,000
7            Caleb Farley           TEN     $4,294,018
8            Caleb Murphy           TEN       $916,666
9           Calvin Ridley           TEN    $10,000,000
10            Cedric Gray           TEN     $1,003,623
11        Chance Campbell           TEN       $915,000
12         Chidobe Awuzie           TEN     $6,940,000
13      Chigoziem Okonkwo           TEN     $1,111,084
14           Cole Spencer           TEN